In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

data_path = Path("./iris.data")
iris_df = pd.read_csv(data_path, header=0, names=["sepal length", "sepal width", "petal length", "petal width", "class"])

iris_df.head()

,sepal length,sepal width,petal length,petal width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [2]:
# prelim data overview:

print(iris_df.describe())
class_no = { "Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica":2 }
binc = np.bincount([class_no.get(c) for c in iris_df["class"]])
print(f"Class counts: {binc}")
print(f"Number of instances: {len(iris_df)} ")

       sepal length  sepal width  petal length  petal width
count    149.000000   149.000000    149.000000   149.000000
mean       5.848322     3.051007      3.774497     1.205369
std        0.828594     0.433499      1.759651     0.761292
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.400000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000
Class counts: [49 50 50]
Number of instances: 149 


In [3]:
import torch
from torch import nn


# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class IrisNetwork(nn.Module):
    def __init__(self):
        super(IrisNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(4, 32),
            nn.ReLU(),
            nn.Linear(32, 3),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = IrisNetwork().to(device)
print(model)

Using cpu device
IrisNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
    (3): ReLU()
  )
)


In [4]:
# define torch.dataset: __init__(), __len__(), __getitem__()
from torch.utils.data import Dataset

class IrisDataSet(Dataset):
    def __init__(self, data_df, transform=None, target_transform=None):
        self.iris_df = data_df
        self.transform = transform
        self.target_transform = target_transform
        self.X = np.asarray(self.iris_df.iloc[:,:4].values, dtype=np.float32)
        self.Y = np.asarray(self.iris_df["class"].values)
    def __len__(self):
        return len(self.Y)
    def __getitem__(self,idx):
        self.x = self.X[idx,:]
        self.y = self.Y[idx]
        if self.transform != None:
            self.x = self.transform(self.x)
        if self.target_transform != None:
            self.y = self.target_transform(self.y)
        return self.x, self.y
            

In [5]:
from sklearn.preprocessing import StandardScaler

def normalize_dataframe(data_df, column_names_to_normalize):
    """
        Normalizes all given columns of a given data frame with a StandardScaler from Sklearn. 
        Input:
            data_df: dataframe with numerical values to normalize
            column_names_to_normalize: list of the names of the columns to be normalized
        Output:
            dataframe with columns normalized
    """
    scaler = StandardScaler()
    data_norm = data_df[column_names_to_normalize].values
    data_normed = scaler.fit_transform(data_norm)
    df_temp = pd.DataFrame(data_normed, columns=column_names_to_normalize, index=data_df.index)
    data_df[column_names_to_normalize]= df_temp
    return data_df
#print(scaler.mean_)
#print(scaler.var_)
#print(f"test normed: {test_df.head()}")

In [6]:
# Dataloading, Normalization:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold

# read data into data-frame:
data_path = Path("./iris.data")
iris_df = pd.read_csv(data_path, header=0, names=["sepal length", "sepal width", "petal length", "petal width", "class"])

# split data into train and test:
train_df, test_df = train_test_split(iris_df, test_size=0.2)
train_df = train_df.copy() # train_df from train_test_split is just a view, i.e. causes problems when normalizing
test_df = test_df.copy()

# normalize the train and test data:
column_names_to_normalize = ["sepal length", "sepal width", "petal length", "petal width"]



train_df = normalize_dataframe(train_df, column_names_to_normalize)
test_df = normalize_dataframe(test_df, column_names_to_normalize)



In [7]:
# Class->Number encoding of the labels:

class_no = { "Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica":2 }
target_transform_class_no = class_no.get


In [8]:
# One Hot Encoding the labels:
from torchvision.transforms import Lambda # might be overkill to call these just for OHE...

# simple OHE encoding:
#class_ohe = { "Iris-setosa": [1,0,0], "Iris-versicolor": [0,1,0], "Iris-virginica":[0,0,1] }
#target_transfrom_class_ohe = class_ohe.get

# another OHE encoding supposing, that the labels y have been number-encoded before:
transform_ohe = Lambda(lambda y: torch.zeros(3, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))


In [9]:
# Target Transform Definition:
from torchvision.transforms import Compose

target_transform_ohe = Compose([
    target_transform_class_no,
    #transform_ohe,
])

In [10]:
# define datasets and dataloaders - WITHOUT K-FOLDING:
from torch.utils.data import DataLoader

train_ds = IrisDataSet(data_df=train_df, target_transform=target_transform_ohe)
test_ds = IrisDataSet(data_df=test_df, target_transform=target_transform_ohe)

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=8, shuffle=True)

In [11]:
# test for datasets and loaders
pred = torch.Tensor()
train_features, train_labels = next(iter(train_dl))
print(train_features.shape)
print(len(train_labels))
print(train_labels.shape)
print(train_labels)
preds = model(train_features.float())
print(preds)
print(preds.argmax(1))

torch.Size([8, 4])
8
torch.Size([8])
tensor([0, 2, 0, 0, 0, 0, 2, 2])
tensor([[0.0000, 0.1726, 0.3914],
        [0.0349, 0.0000, 0.0000],
        [0.0000, 0.6260, 0.6933],
        [0.0000, 0.1345, 0.3755],
        [0.0000, 0.0079, 0.1712],
        [0.0000, 0.1148, 0.3112],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
tensor([2, 0, 2, 2, 2, 2, 0, 0])


In [12]:
# test and train loops:

def train_loop(dataloader, model, loss_fn, optimizer):
    data_size = len(dataloader.dataset)
    for n_batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if n_batch % 5 == 0:
            print(f"\n Batch Loss: {loss.item()} --- Current Sample: {n_batch * len(y)} / {data_size}")

def test_loop(dataloader, model, loss_fn):
    data_size = len(dataloader.dataset)
    losses, correct = 0, 0
    
    with torch.no_grad():
        for (X,y) in dataloader:
            pred = model(X)
            losses += loss_fn(pred, y).item()
            correct += (pred.argmax(1)== y).type(torch.float).sum().item()
    
    print(f"\n Avg. Test Loss per Epoch: {losses/ data_size :.8f}")
    print(f"\n Accuracy per Epoch: {correct / data_size: .4f}")
    

In [13]:
# Train the model:
from torch.utils.data import DataLoader

learning_rate = 1e-8
epochs = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)
apply_stratified_kfold = True

for ep in range(epochs):
    print(f"\n----- Epoch: {ep} -----")
    if apply_stratified_kfold:
        column_names_to_normalize = ["sepal length", "sepal width", "petal length", "petal width"]
        X = iris_df[column_names_to_normalize]
        y = iris_df["class"].values
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        for fold, (train_ids, test_ids) in enumerate(skf.split(X, y)):
            print(f"Fold: {fold}")
            train_df = iris_df.iloc[train_ids].copy()
            train_df = normalize_dataframe(train_df, column_names_to_normalize)
            test_df = iris_df.iloc[test_ids].copy()
            test_df = normalize_dataframe(test_df, column_names_to_normalize)
            train_ds = IrisDataSet(data_df=train_df, target_transform=target_transform_ohe)
            test_ds = IrisDataSet(data_df=test_df, target_transform=target_transform_ohe)
            train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
            test_dl = DataLoader(test_ds, batch_size=8, shuffle=True)

            model.train()
            train_loop(train_dl, model, loss_fn, optimizer)
            model.eval()
            test_loop(test_dl, model, loss_fn)
    else:
        model.train()
        train_loop(train_dl, model, loss_fn, optimizer)
        model.eval()
        test_loop(test_dl, model, loss_fn)
print("----- Finished Training -----")


----- Epoch: 0 -----
Fold: 0

 Batch Loss: 1.1644858121871948 --- Current Sample: 0 / 119

 Batch Loss: 1.1123229265213013 --- Current Sample: 40 / 119

 Batch Loss: 1.1150720119476318 --- Current Sample: 80 / 119

 Avg. Test Loss per Epoch: 0.15326178

 Accuracy per Epoch:  0.0333
Fold: 1

 Batch Loss: 1.1029298305511475 --- Current Sample: 0 / 119

 Batch Loss: 1.136976957321167 --- Current Sample: 40 / 119

 Batch Loss: 1.1158716678619385 --- Current Sample: 80 / 119

 Avg. Test Loss per Epoch: 0.15367104

 Accuracy per Epoch:  0.0000
Fold: 2

 Batch Loss: 1.1636959314346313 --- Current Sample: 0 / 119

 Batch Loss: 1.1201081275939941 --- Current Sample: 40 / 119

 Batch Loss: 1.1226904392242432 --- Current Sample: 80 / 119

 Avg. Test Loss per Epoch: 0.15302275

 Accuracy per Epoch:  0.1000
Fold: 3

 Batch Loss: 1.1364773511886597 --- Current Sample: 0 / 119

 Batch Loss: 1.1371252536773682 --- Current Sample: 40 / 119

 Batch Loss: 1.173823595046997 --- Current Sample: 80 / 119

